**Project 4** - Group 2 (Emma, Gabriel, Ben, Junhao)

<br>

<font size="6">Part 2: Modelling

## Load Libraries / Import Datasets

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic

from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline

from sklearn.metrics import (
    confusion_matrix, 
    plot_confusion_matrix,
    accuracy_score,
    plot_roc_curve,
    roc_auc_score,
    recall_score,
    precision_score,
    f1_score
)

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
from xgboost import XGBClassifier

pd.set_option('display.max_colwidth', 0)

In [11]:
train_weather = pd.read_csv(r'../datasets/train_weather_v3.csv')
test_weather = pd.read_csv(r'../datasets/test_weather_v3.csv')
submission = pd.read_csv(r'../datasets/sampleSubmission.csv')

The cleaned test dataset has ~13 times more entries than the train dataset.

In [12]:
train_weather.shape

(8475, 28)

In [13]:
test_weather.shape

(116293, 28)

## Functions

In [35]:
def get_model(model, gridcv, pipe_params={}):
    '''Function to apply GriSearchCV based on selected model and parameters'''
    
    #Scale X
    ss = StandardScaler()
    Xsc_train = ss.fit_transform(X_train)
    Xsc_test = ss.transform(X_test)
    
    #Models to test
    models = {'lr': LogisticRegression(),
              'nb': MultinomialNB(),
              'rf': RandomForestClassifier(),
              'et': ExtraTreesClassifier(),
              'ada': AdaBoostClassifier(base_estimator=DecisionTreeClassifier()),
              'knn': KNeighborsClassifier(),
              'gb': GradientBoostingClassifier(),
              'xgb': xgb.XGBClassifier()
             }
    
    #pipeline to include SMOTE since this is an unbalanced dataset 
    pipe = imbpipeline([
        ('sm', SMOTE(random_state=42)), 
        (model, models[model]),
            ])
    
    pipe_params = pipe_params
    
    #GriSesarchCV on user assigned parameters
    gridsearch = GridSearchCV(pipe, pipe_params, cv=gridcv, scoring='roc_auc', verbose=1, n_jobs=-1)
    
    gridsearch.fit(Xsc_train, y_train)
    
    #predict on test 
    preds = gridsearch.predict(Xsc_test)
    tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

    model_probs_train = gridsearch.predict_proba(Xsc_train)[:, 1]
    model_probs_test = gridsearch.predict_proba(Xsc_test)[:, 1]
    roc_auc = cross_val_score(pipe, Xsc_train, y_train, scoring='roc_auc', cv=gridcv)
    best_params = gridsearch.best_params_
    
    #Metrics to produce
    summary ={
        'model': model,
        'accuracy': accuracy_score(y_test, preds),
        'specificity': tn / (tn+fp),
        'recall': recall_score(y_test, preds),
        'precision': precision_score(y_test, preds),
        'f1_score': f1_score(y_test, preds),
        'AUC_CV': roc_auc.mean(),
        'AUC_train': roc_auc_score(y_train, model_probs_train),
        'AUC_test': roc_auc_score(y_test, model_probs_test),
        'best params': best_params}
    
    summary_df = pd.DataFrame.from_dict(summary, orient='Index', columns=[str(model)])

    return summary_df

## Modelling

In [38]:
#Train/Test(aka Validation) split on Train dataset 

X = train_weather.drop(columns=['WnvPresent'])
y = train_weather['WnvPresent'] 
    
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

metrics = pd.DataFrame(index=[0])

### Baseline 

For the baseline model, we run it a logistic regression model with no parameter tuning. It produced a ROC_AUC test score of 0.81. 

In [39]:
get_model('lr', 5, pipe_params={})

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

,lr
model,lr
accuracy,0.73714
specificity,0.735661
recall,0.763158
precision,0.141005
f1_score,0.23803
AUC_CV,0.833047
AUC_train,0.844657
AUC_test,0.817988
best params,{}


### Logistic Regression

In [40]:
%%time

lr_results = get_model('lr', 5, pipe_params={
    'lr__C' : np.logspace(0.01, 1, 10),
    'lr__solver': ['sag','saga', 'liblinear'],    
    'lr__max_iter': [1000, 5000, 10000]
})

lr_results

Fitting 5 folds for each of 90 candidates, totalling 450 fits


/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

CPU times: user 4.81 s, sys: 181 ms, total: 4.99 s
Wall time: 4min 1s


/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

,lr
model,lr
accuracy,0.73714
specificity,0.735661
recall,0.763158
precision,0.141005
f1_score,0.23803
AUC_CV,0.833047
AUC_train,0.844654
AUC_test,0.817984
best params,"{'lr__C': 1.023292992280754, 'lr__max_iter': 1000, 'lr__solver': 'saga'}"


### Random Forest

In [151]:
%%time

rf_results = get_model('rf', 5, pipe_params={
    'rf__n_estimators': [100, 150],
    'rf__min_samples_split': [2, 3, 5],
    'rf__min_samples_leaf': [2, 5, 10]
})

rf_results

Fitting 5 folds for each of 18 candidates, totalling 90 fits
CPU times: user 8.7 s, sys: 101 ms, total: 8.8 s
Wall time: 25.2 s


,rf
model,rf
accuracy,0.880604
specificity,0.906733
recall,0.421053
precision,0.204255
f1_score,0.275072
AUC_CV,0.756269
AUC_train,0.926258
AUC_test,0.824266
best params,"{'rf__min_samples_leaf': 10, 'rf__min_samples_split': 3, 'rf__n_estimators': 100}"


### K Nearest Neighbors

In [153]:
%%time

knn_results = get_model('knn', 5, pipe_params={
    'knn__leaf_size': [10, 20, 30],
    'knn__n_neighbors': [10, 25, 50, 75],
    'knn__p': [1, 2]
})

knn_results

Fitting 5 folds for each of 24 candidates, totalling 120 fits
CPU times: user 8.94 s, sys: 1.36 s, total: 10.3 s
Wall time: 16.3 s


,knn
model,knn
accuracy,0.746579
specificity,0.74813
recall,0.719298
precision,0.139693
f1_score,0.233951
AUC_CV,0.729757
AUC_train,0.904566
AUC_test,0.784709
best params,"{'knn__leaf_size': 10, 'knn__n_neighbors': 50, 'knn__p': 1}"


### GradientBoost

In [44]:
%%time

gb_results = get_model('gb', 5, pipe_params={
    'gb__max_depth': [2,5,8],
    'gb__n_estimators': [25, 50],
    'gb__learning_rate': [.05, .1, .15],
    'gb__min_samples_leaf':[30, 50],
    'gb__min_samples_split':[30, 50],
})

gb_results

Fitting 5 folds for each of 108 candidates, totalling 540 fits
CPU times: user 16.1 s, sys: 390 ms, total: 16.5 s
Wall time: 4min 21s


,gb
model,gb
accuracy,0.899953
specificity,0.926185
recall,0.438596
precision,0.252525
f1_score,0.320513
AUC_CV,0.837753
AUC_train,0.928655
AUC_test,0.838087
best params,"{'gb__learning_rate': 0.05, 'gb__max_depth': 8, 'gb__min_samples_leaf': 50, 'gb__min_samples_split': 30, 'gb__n_estimators': 100}"


### AdaBoost

In [9]:
%%time

ada_results = get_model('ada', 5, pipe_params={
    'sm__k_neighbors': [20, 50, 80], 
    'ada__n_estimators': [30, 50, 80],
    'ada__learning_rate': [0.1, 0.3, 0.7, 1],
    'ada__base_estimator__max_depth': [1, 2] 
})

ada_results

Fitting 5 folds for each of 72 candidates, totalling 360 fits
CPU times: user 12.4 s, sys: 209 ms, total: 12.6 s
Wall time: 1min 9s


,ada
model,ada
accuracy,0.934403
specificity,0.981047
recall,0.114035
precision,0.254902
f1_score,0.157576
AUC_CV,0.732466
AUC_train,0.893912
AUC_test,0.830113
best params,"{'ada__base_estimator__max_depth': 2, 'ada__learning_rate': 0.3, 'ada__n_estimators': 80, 'sm__k_neighbors': 50}"


### XGBoost

In [36]:
xgb_results = get_model('xgb', 5, pipe_params={
    'xgb__use_label_encoder' : [False],
    'xgb__subsample' : [1, 0.5],
    'xgb__colsample_bytree' : [1, 0.75],
    'xgb__learning_rate' : [0.17, 0.1],
    'xgb__min_child_weight': [1, 3],
    'xgb__max_depth' : [3],
    'xgb__n_estimators' : [300, 100],
    'xgb__reg_alpha' : [5, 1, 10],
    'xgb__reg_lambda' : [35, 10, 50],
    'xgb__gamma' : [0.05, 0.1],
    'xgb__objective' : ['binary:logistic']})

xgb_results

Fitting 5 folds for each of 576 candidates, totalling 2880 fits
[13:34:07] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:34:08] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:34:09] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:34:10] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:34:11] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,xgb
model,xgb
accuracy,0.873997
specificity,0.895761
recall,0.491228
precision,0.211321
f1_score,0.295515
AUC_CV,0.833137
AUC_train,0.903746
AUC_test,0.839701
best params,"{'xgb__colsample_bytree': 1, 'xgb__gamma': 0.1, 'xgb__learning_rate': 0.17, 'xgb__max_depth': 3, 'xgb__min_child_weight': 1, 'xgb__n_estimators': 300, 'xgb__objective': 'binary:logistic', 'xgb__reg_alpha': 10, 'xgb__reg_lambda': 50, 'xgb__subsample': 0.5, 'xgb__use_label_encoder': False}"


## Summary

In [37]:
overall_df = pd.concat([lr_results, rf_results, knn_results, gb_results, ada_results], axis=1).T
overall_df.sort_values(ascending=False, by='AUC_test').drop(columns=['model'])

NameError: name 'lr_results' is not defined

### ROC-AUC curve

### Confusion Matrix

### Prediction vs True

### Coefficient Analysis

## Kaggle Model

In [20]:
#Train/Test split on full dataset

X_full_train = train_weather.drop(columns=['WnvPresent'])
X_full_test = test_weather.drop(columns=['Id'])
y_full_train = train_weather['WnvPresent']


In [15]:
%%time

ss = StandardScaler()
X_full_train_sc = ss.fit_transform(X_full_train)
X_full_test_sc = ss.transform(X_full_test)

#Pipeline
pipe = imbpipeline([
                ('sm', SMOTE(random_state=42)), 
                ('lr', LogisticRegression()),
            ])

pipe_params = {
    'lr__C' : [1.023292992280754],
    'lr__solver' : ['saga'],
    'lr__max_iter' : [1000]
}
    
gridsearch = GridSearchCV(pipe, pipe_params, cv=5, scoring='roc_auc', verbose=1, n_jobs=-1)

#Fit model
gridsearch.fit(X_full_train_sc, y_full_train)
    
preds = gridsearch.predict_proba(X_full_test_sc)


model_probs_train = gridsearch.predict_proba(X_full_train_sc)[:, 1]
model_probs_test = gridsearch.predict_proba(X_full_test_sc)[:, 1]
roc_auc = cross_val_score(pipe, X_full_train_sc, y_full_train, scoring='roc_auc', cv=5)
    
print('model', 'lr')
print('AUC_CV', roc_auc.mean())
print('AUC_train', roc_auc_score(y_full_train, model_probs_train))
print('best_params', gridsearch.best_params_)


Fitting 5 folds for each of 60 candidates, totalling 300 fits


/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

model lr
AUC_CV 0.7573432061608809
AUC_train 0.8393473819573356
best_params {'lr__C': 1.023292992280754, 'lr__max_iter': 1000, 'lr__solver': 'saga'}
0.7575306848641691
CPU times: user 7.16 s, sys: 208 ms, total: 7.37 s
Wall time: 3min 57s


/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/benjaminpoh/opt/anaconda3/envs/dsi24/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

In [284]:
# pipe = imbpipeline([
#                 ('ss', StandardScaler()),
#                 ('sm', SMOTE(random_state=42)), 
#                 ('rf', RandomForestClassifier()),
#             ])

# pipe_params = {
#     'rf__C': np.logspace(0.01, 1, 5),
#     'rf__min_samples_split': [2, 3],
#     'rf__min_samples_leaf': [2, 5, 10]
# }
    
# gridsearch = GridSearchCV(pipe, pipe_params, cv=5, scoring='roc_auc', verbose=1, n_jobs=-1)
    
# gridsearch.fit(X_full_train, y_full_train)
    
# preds = gridsearch.predict_proba(X_full_test)


# model_probs_train = gridsearch.predict_proba(X_full_train)[:, 1]
# model_probs_test = gridsearch.predict_proba(X_full_test)[:, 1]
# roc_auc = cross_val_score(pipe, X_full_train, y_full_train, scoring='roc_auc', cv=5)
    
# print( 'model', 'ada')
# print('AUC_CV', roc_auc.mean())
# print('AUC_train', roc_auc_score(y_full_train, model_probs_train))
# print('best_params', gridsearch.best_params_)

# print(gridsearch.best_score_)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
model ada
AUC_CV 0.41853426857811193
AUC_train 0.920408157138779
best_params {'rf__min_samples_leaf': 10, 'rf__min_samples_split': 2, 'rf__n_estimators': 150}
0.5626193360540922


In [21]:
ss = StandardScaler()
X_full_train_sc = ss.fit_transform(X_full_train)
X_full_test_sc = ss.transform(X_full_test)

pipe = imbpipeline([
                ('sm', SMOTE(random_state=42)), 
                ('gb', GradientBoostingClassifier()),
            ])

pipe_params = {
    'gb__max_depth': [2,5,8],
    'gb__n_estimators': [100, 125, 150],
    'gb__learning_rate': [.05, .1, .15],
    'gb__min_samples_leaf':[1,2,3],
    'gb__min_samples_split':[2,3,5],
}
    
gridsearch = GridSearchCV(pipe, pipe_params, cv=5, scoring='roc_auc', verbose=1, n_jobs=-1)
    
gridsearch.fit(X_full_train_sc, y_full_train)
    
preds = gridsearch.predict_proba(X_full_test_sc)


model_probs_train = gridsearch.predict_proba(X_full_train_sc)[:, 1]
model_probs_test = gridsearch.predict_proba(X_full_test_sc)[:, 1]
roc_auc = cross_val_score(pipe, X_full_train_sc, y_full_train, scoring='roc_auc', cv=5)
    
print('model', 'lr')
print('AUC_CV', roc_auc.mean())
print('AUC_train', roc_auc_score(y_full_train, model_probs_train))
print('best_params', gridsearch.best_params_)

print(gridsearch.best_score_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
model lr
AUC_CV 0.6214246189960975
AUC_train 0.8328218565121257
best_params {'gb__learning_rate': 0.05, 'gb__max_depth': 2, 'gb__min_samples_leaf': 1, 'gb__min_samples_split': 2, 'gb__n_estimators': 100}
0.671607943391864


In [22]:
ss = StandardScaler()
X_full_train_sc = ss.fit_transform(X_full_train)
X_full_test_sc = ss.transform(X_full_test)

pipe = imbpipeline([
                ('sm', SMOTE(random_state=42)), 
                ('ada', AdaBoostClassifier()),
            ])

pipe_params = {
    'sm__k_neighbors': [20, 50, 80], 
    'ada__n_estimators': [30, 50, 80],
    'ada__learning_rate': [0.1, 0.3, 0.7, 1],
    'ada__base_estimator__max_depth': [1, 2] 
}
    
gridsearch = GridSearchCV(pipe, pipe_params, cv=5, scoring='roc_auc', verbose=1, n_jobs=-1)
    
gridsearch.fit(X_full_train_sc, y_full_train
               


Fitting 5 folds for each of 72 candidates, totalling 360 fits


AttributeError: 'NoneType' object has no attribute 'set_params'

In [ ]:
preds = gridsearch.predict_proba(X_full_test_sc)


model_probs_train = gridsearch.predict_proba(X_full_train_sc)[:, 1]
model_probs_test = gridsearch.predict_proba(X_full_test_sc)[:, 1]
roc_auc = cross_val_score(pipe, X_full_train_sc, y_full_train, scoring='roc_auc', cv=5)
    
print('model', 'lr')
print('AUC_CV', roc_auc.mean())
print('AUC_train', roc_auc_score(y_full_train, model_probs_train))
print('best_params', gridsearch.best_params_)

print(gridsearch.best_score_)

In [6]:
predict_y=pd.DataFrame(preds[:,1])
predict_y =predict_y.rename(columns={0:'WnvPresent'})

In [8]:
submission = pd.read_csv(r'../datasets/sampleSubmission.csv')
submission['WnvPresent'] = predict_y['WnvPresent']

In [10]:
submission.to_csv(r'../datasets/sampleSubmission_v12.csv', index=False)

In [21]:
submission

,Id,WnvPresent
0,1,0.132141
1,2,0.051930
2,3,0.057630
3,4,0.000066
4,5,0.000071
...,...,...
116288,116289,0.000004
116289,116290,0.000004
116290,116291,0.000005
116291,116292,0.001593
